##Importing 라이브러리

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

seed_everything(42)  # 무조건 42로 세팅!!

In [2]:
!pip install xgboost

In [3]:
from xgboost import XGBClassifier

##파일 불러오기

In [106]:
train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")
submission = pd.read_csv("/content/sample_submission.csv")
print(train.shape, test.shape, submission.shape)

(48199, 13) (7695, 12) (7695, 2)


##데이터 전처리 (Train + Test)

###0)불필요 열 제거

In [107]:
train.drop(columns=['click_id', 'session ID'], inplace=True)

In [108]:
test.drop(columns=['click_id', 'session ID'], inplace=True)

###1) main category 결측치 채우기

In [109]:
#main category 결측치 채우기 --> by clothing model의 앞 문자(A:1.0, B:2.0, C:3.0, P:4.0)
mask = (train["page 1 (main category)"].isna()) #& (train["page 2 (clothing model)"].notna()) &(train["country"] == 12)
# page 2 (clothing model)의 첫 번째 문자 가져와서 매핑 (A -> 1.0, B -> 2.0, C -> 3.0, P -> 4.0)
mapping = {'A': 1.0, 'B': 2.0, 'C': 3.0, 'P': 4.0}
# 변환 적용
train.loc[mask, "page 1 (main category)"] = train.loc[mask, "page 2 (clothing model)"].str[0].map(mapping)
# 변환이 적용된 데이터 개수 확인
print(train.loc[mask, ["page 1 (main category)", "page 2 (clothing model)"]].dropna().shape[0])
#page 2 (clothing model) 삭제
train.drop(columns=['page 2 (clothing model)'], inplace=True)

31


In [110]:
#main category 결측치 채우기 --> by clothing model의 앞 문자(A:1.0, B:2.0, C:3.0, P:4.0)
mask = (test["page 1 (main category)"].isna()) #& (train["page 2 (clothing model)"].notna()) &(train["country"] == 12)
# page 2 (clothing model)의 첫 번째 문자 가져와서 매핑 (A -> 1.0, B -> 2.0, C -> 3.0, P -> 4.0)
mapping = {'A': 1.0, 'B': 2.0, 'C': 3.0, 'P': 4.0}
# 변환 적용
test.loc[mask, "page 1 (main category)"] = test.loc[mask, "page 2 (clothing model)"].str[0].map(mapping)
# 변환이 적용된 데이터 개수 확인
print(test.loc[mask, ["page 1 (main category)", "page 2 (clothing model)"]].dropna().shape[0])
#page 2 (clothing model) 삭제
test.drop(columns=['page 2 (clothing model)'], inplace=True)

3


###2)시간과 관련한 데이터 추출

In [111]:
train['date'] = pd.to_datetime(train['date'])
train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month
train['day'] = train['date'].dt.day
train['weekday'] = train['date'].dt.weekday
train.drop(columns=['date'], inplace=True)

In [112]:
test['date'] = pd.to_datetime(test['date'])
test['year'] = test['date'].dt.year
test['month'] = test['date'].dt.month
test['day'] = test['date'].dt.day
test['weekday'] = test['date'].dt.weekday
test.drop(columns=['date'], inplace=True)

###3)colour

In [113]:
# 색상 그룹 매핑 (높은 Click 비율 → 낮은 숫자)
color_map = {
    2: 0, 3: 0, 8: 0,  # 가장 높은 Click 비율 (가장 낮은 숫자)
    4: 1, 7: 1, 14: 1,  # 두 번째 그룹
    9: 2, 10: 2, 11: 2, 13: 2,  # 세 번째 그룹
    1: 3, 5: 3, 6: 3, 12: 3  # 가장 낮은 Click 비율 (가장 높은 숫자)
}

# colour 컬럼 변환
train["colour"] = train["colour"].map(color_map)

In [114]:
test["colour"] = test["colour"].map(color_map)

###country target encoding

In [115]:
train["country_encoded"] = train.groupby("country")["Clicked"].transform("mean")

In [117]:
test["country_encoded"] = train.groupby("country")["Clicked"].transform("mean")

##학습하기
-XGboost 모델 사용하기
-하이퍼파라미터의 영향을 많이 받는 모델이기에 최적의 조합 찾기

In [118]:
X = train.drop('Clicked', axis=1)
y = train['Clicked']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(38559, 13) (9640, 13) (38559,) (9640,)


-n_estimators=100, max_depth=3 : [0.6816-0.6701] with 결측치 처리 & drop

-

In [66]:
clf=XGBClassifier()
params = {'n_estimators':[50, 100, 200],
          'max_depth':[2,3,4],
          }
grid_cv = GridSearchCV(clf, param_grid=params, cv=3, n_jobs=-1)
grid_cv.fit(X_train, y_train)
print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

최적 하이퍼 파라미터:
 {'max_depth': 2, 'n_estimators': 100}
최고 예측 정확도: 0.6738


In [ ]:
#colour를 구간으로 조합함 & country 클릭률에 따라 인코딩 안함
#Train ACC : 0.7055
#Val ACC : 0.6773

#country 클릭률에 따라 인코딩 O
#Train ACC : 0.7036
#Val ACC : 0.6794

In [119]:
#xgb_clf = XGBClassifier(**grid_cv.best_params_)
#[0.6939-0.6647],[]
xgb_clf = XGBClassifier(n_estimators=100, max_depth=4, learning_rate = 0.05)
xgb_clf.fit(X_train, y_train)

train_pred = xgb_clf.predict(X_train)
val_pred = xgb_clf.predict(X_val)

print("Train ACC : %.4f" % accuracy_score(y_train, train_pred))
print("Val ACC : %.4f" % accuracy_score(y_val, val_pred))

Train ACC : 0.6844
Val ACC : 0.6790


In [120]:
feature_importance = pd.DataFrame(xgb_clf.feature_importances_.reshape((1, -1)), columns=X_train.columns, index=['feature_importance'])
feature_importance

order   country  page 1 (main category)   colour  \
feature_importance  0.135794  0.016528                0.164444  0.33877   

                    location  model photography      page     price  year  \
feature_importance  0.105333           0.053884  0.095511  0.043394   0.0   

                    month       day   weekday  country_encoded  
feature_importance    0.0  0.015012  0.012627         0.018702

Train ACC : 0.6793
Val ACC : 0.6686


##submission.csv 생성

In [121]:
X_test = test

output = xgb_clf.predict(X_test)
assert len(output) == 7695  # sanity check

In [122]:
submission['Clicked'] = output
submission.to_csv('submission.csv', index=False)